In [91]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np

# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path)


In [92]:
# function for ratings scrape

def user_ratings(username):
    ratings_count = 0
    user_ratings = []
    avail_results = True 
    while avail_results:
        # assemble the url, make the request, isolate table
        url = f"https://www.beeradvocate.com/user/beers/?start={ratings_count}&ba={username}&order=dateD&view=R"
        html = requests.get(url)
        print(html.url)
        soup = BeautifulSoup(html.text, 'html.parser')
         # number of user ratings 
        ######need to dump the ratings from user scrape
        
        results = soup.find_all('table')
        

        #increase ratings count to get next page
        ratings_count += 50
        
        #get correct table rows
        ratings_list = results[0].find_all('tr')[3:] #increase ratings count to get next page
        
        #####need catch for no user ratings
        if len(ratings_list) > 0:
        
            #loop through and sort info
            for tr in ratings_list:
                tds = tr.find_all('td')

                # isolate some info
                user_rating = tds[-2].text
#                 abv = tds[-3].text
                rDev = re.findall(r"([\d.]*\d+)", tds[-1].text)[0]
                # further isolate element in list holding 3 features
                add_beer_info = tds[2].find_all('a')
                
                # extract info
#                 beer_name = add_beer_info[0].text
                beer_link_total = add_beer_info[0]['href']
                base_url_ba = "https://www.beeradvocate.com"
                beer_link = base_url_ba + beer_link_total[0:beer_link_total.find("?")]
#                 beer_style = add_beer_info[2].text
#                 brewery = add_beer_info[1].text

                #dump into dict
                ratings_dict = {
#                     "beer_name": beer_name,
#                     "brewery": brewery,
#                     "beer_style": beer_style,
#                     "beer_style_match": re.sub(r'\W+', '', beer_style.lower()),
                    "beer_link": beer_link,
                    "user_rating": user_rating,
#                     "abv": abv,
                    "rDev": rDev

                }

                # append link to list
                user_ratings.append(ratings_dict)
        else: 
            
            avail_results = False
    user_ratings_df = pd.DataFrame(user_ratings).drop_duplicates()
    
    return user_ratings_df

In [1]:
#user_ratings_df = user_ratings('brutaltruth')

In [16]:
# beerFeatures_df = pd.read_csv("../StyleLinkScrape&Results/styleFeaturesComplete.csv")
# beerFeatures_df

Color  Flavor  Hoppiness  Maltiness  Fruitiness  Body  Sweetness  \
0    pale       1          0          0           0     1          0   
1    gold       2          2          2           0     1          0   
2    pale       2          2          0           0     2          0   
3    gold       2          3          2           0     2          0   
4   amber       2          0          2           0     2          0   
5   amber       2          1          0           0     3          0   
6   black       1          0          1           0     2          1   
7   amber       2          2          2           0     2          2   
8   amber       3          1          3           0     2          2   
9   dark        3          0          2           0     3          0   
10   gold       2          0          2           3     2          0   
11   gold       2          2          0           0     2          0   
12  dark        2          2          2           0     2          0   
13  dark        2          1          2           0     3          2   
14  amber       2          2          2           1     2          0   
15  amber       2          2          2           0     2          0   
16   gold       2          1          1           1     1          0   
17   gold       2          2          0           2     1          0   
18   gold       2          2          2           2     2          0   
19  amber       2          3          2           0     2          0   
20  amber       3          3          3           0     2          0   
21   gold       2          2          2           0     2          0   
22  dark        2          1          2           0     2          2   
23  dark        1          2          0           0     2          1   
24  dark        3          0          0           0     2          0   
25  black       2          1          1           0     2          0   
26  black       2          2          2           0     3          2   
27  black       2          2          1           0     2          1   
28  black       2          0          2           0     3          2   
29  black       2          1          2           0     1          0   
30   pale       2          2          1           0     2          1   
31   gold       3          0          2           2     2          1   
32   gold       2          1          3           1     1          2   
33  amber       3          0          3           2     3          2   
34  dark        3          1          2           2     2          0   
35   gold       2          0          0           3     2          1   
36   pale       2          1          1           1     2          0   
37   pale       2          0          0           0     2          0   
38   gold       2          0          0           2     2          0   
39  amber       2          0          2           2     2          1   
40   gold       2          1          0           2     2          0   
41   gold       3          0          0           0     2          0   
42  amber       2          1          1           3     2          2   
43  amber       2          0          2           2     1          0   
44   gold       2          0          0           3     2          0   
45  amber       2          0          2           0     2          0   
46  amber       2          1          2           0     2          2   
47  dark        3          0          2           0     2          0   

   Custom Flavor  Sour    Aroma  spicy  bitter                     csv_name  \
0              0     0        0      0       0         americanadjunctlager   
1              0     0        0      2       0            munichhelleslager   
2              0     0   floral      0       2               germanpilsener   
3              0     0   floral      0       2                czechpilsener   
4              0     0  caramel      0       0        americanamberredlager   
5

In [18]:
def match_styles(username):
    #create df of user ratings
    user_ratings_df = user_ratings(username)
    print(len(user_ratings_df))
    # import or query for styles_df
#     beer_features_df = pd.read_csv("../encoded1.csv")
# kxa moved this to current folder
    beer_features_df = pd.read_csv("encoded1.csv")
    user_ratings_df = user_ratings_df.merge(beer_features_df, left_on = 'beer_link', right_on = 'URL', how = "right")
    rec_df = user_ratings_df[user_ratings_df['beer_link'].isnull()]
    user_df = user_ratings_df[user_ratings_df['beer_link'].notnull()]
    return rec_df, user_df


In [19]:
member = 'clockworkorange'
red_df, user_df = match_styles(member)


https://www.beeradvocate.com/user/beers/?start=0&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=50&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=100&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=150&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=200&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=250&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=300&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=350&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=400&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=450&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate.com/user/beers/?start=500&ba=clockworkorange&order=dateD&view=R
https://www.beeradvocate

In [20]:
len(user_df)

558

In [21]:
len(red_df)

20688

In [31]:
# determine what is good and what is bad
# convert rating to numeric
user_df['user_rating'] = pd.to_numeric(user_df['user_rating'])

In [43]:
# user_df.info()
# user_df[user_df['user_rating']>3.5]
bins = [1,2,3,4,5.1]
labels = ['1-1.99', '2-2.99','3-3.99','4-5']
user_df['bin'] = pd.cut(user_df['user_rating'],bins=bins, labels=labels)

In [44]:
user_df['bin'].value_counts()

3-3.99    366
4-5       179
2-2.99     13
1-1.99      0
Name: bin, dtype: int64

In [59]:
fourAndAbove = user_df['beer_link'][user_df['bin']=='4-5'].count()
fourAndAbove

179

In [72]:
# this is how many records = 5% of the count of ratings
fivePercentCount = int(len(user_df) * .05)

In [ ]:
# need to do a check here - if fiverPercentCount is less than ? then we need to do something else

In [68]:
#sort by user_rating descending and then reindex
user_sort = user_df.sort_values(by='user_rating', ascending = False)
user_sort = user_sort.reset_index(drop=True)

In [97]:
# pull in the file instead of rerunning
user_sort = pd.read_csv("clockworkorangesortmatch.csv", encoding = "ISO-8859-1")
#get the rating score at the 5 percent mark
tenPercentRating = user_sort['user_rating'].iloc[53]
user_sort['good'] = np.where(user_sort['user_rating']>=tenPercentRating, 'good','bad')
user_sort.to_csv(member+'sortmatch2.csv', index = False)

In [73]:
#get the rating score at the 5 percent mark
fivePercentRating = user_sort['user_rating'].iloc[fivePercentCount-1]

In [ ]:
# we can add something her to 

In [85]:
# user_sort[user_sort['user_rating']<fivePercentRating]

In [75]:
def goodBad(row):
    if row['user_rating'] >= fivePercentRating:
        val = 'good'
    else:
        val = 'bad'
    return val

In [78]:
# apply good or bad depending on 
user_sort['good'] = user_sort.apply(goodBad, axis=1)

In [ ]:
#here's another way to set goodbad
user_sort['goodtest'] = np.where(user_sort['user_rating']>=fivePercentRating, 'good','bad')

In [90]:
user_sort.to_csv(member+'sortmatch.csv', index = False)

In [82]:
# user_sort.iloc[30]

In [24]:
# test = user_ratings_df.merge(beer_features, left_on = 'beer_link', right_on = 'URL', how = "right")

In [36]:
user_df.columns

Index(['beer_link', 'rDev', 'user_rating', 'ABV', 'Availaility', 'Brewer',
       'BrosScore', 'ImageLink', 'Location', 'Name', 'Ranking', 'RatingCount',
       'ReviewCount', 'Score', 'ScoreClass', 'Style', 'URL', 'pDev', 'Flavor',
       'Hoppiness', 'Maltiness', 'Fruitiness', 'Body', 'Sweetness', 'Sour',
       'spicy', 'bitter', 'Aroma_None', 'Aroma_caramel', 'Aroma_floral',
       'Aroma_fruit', 'Custom Flavor_None', 'Custom Flavor_banana',
       'Custom Flavor_brown sugar', 'Custom Flavor_burnt',
       'Custom Flavor_caramel', 'Custom Flavor_chocolate',
       'Custom Flavor_earthy', 'Custom Flavor_herbal', 'Custom Flavor_pumpkin',
       'Custom Flavor_roasted', 'Custom Flavor_smoke', 'Custom Flavor_toasted',
       'Custom Flavor_wheat', 'Color_amber', 'Color_black', 'Color_dark ',
       'Color_gold', 'Color_pale'],
      dtype='object')

In [26]:
# len(test)

In [25]:
# recList = test[test['beer_link'].isnull()]
# len(recList)

In [24]:
# user_match = test[test['beer_link'].notnull()]
# len(user_match)

In [22]:
# user_df.to_csv('../stonedtrippinmatch.csv', index = False)
user_df.to_csv(member+'match.csv', index = False)

In [23]:
red_df.to_csv(member+'recs.csv', index = False)

In [35]:
# user_match.sort_values('user_rating', ascending = False, inplace=True)
# top = user_match.head(35)

/Users/ronessaacquesta/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
# df.to_csv('brutaltruthsbeers.csv')

In [2]:
# top

In [3]:
# a_list

In [4]:
# username = 'brutaltruth'
# ratings_count = 0 
# url = f"https://www.beeradvocate.com/user/beers/?start={ratings_count}&ba={username}&order=dateD&view=R"

# html = requests.get(url)


# #     html = browser.html
#  # Parse HTML with Beautiful Soup
# soup = BeautifulSoup(html.text, 'html.parser')
# results = soup.find_all('table')
#     # results
   
#     # Click the 'Next' button on each page
#    # browser.click_link_by_partial_text('next')

In [5]:
# a_list = results[0].find_all('a')
# del a_list[:11]
# del a_list[-4:]
# a_list



In [6]:
# a_item_list=[]
# for x in a_list:
#     a_item=x.text
#     a_item_list.append(a_item)
# len(a_item_list)



In [7]:
# a_url_list=[]
# for x in a_list:
#     a_item=x['href']
#     a_url_list.append(a_item)
# a_url_list

# beer_url_list=[]
# for item in a_url_list[::3]:
#     beer_url_list.append(item)
# beer_url_list

In [8]:
# for item in a_item_list[::3]:
#     beer_name_list.append(item)
# print(beer_name_list)


# for item in a_item_list[1::3]:
#     brewry_list.append(item)
# #print(brewry_list)


# for item in a_item_list[2::3]:
#     beer_sty_list.append(item)
# #print(beer_sty_list)


In [35]:
# len(beer_name_list)
# len(brewry_list)
# len(beer_sty_list)

In [9]:
# b_list = results[0].find_all('b')
# del b_list[-1:]
# del b_list[:1]
# # b_list

# b_item_list=[]
# for item in b_list:
#     b_item=item.text
#     b_item_list.append(b_item)
# len(b_item_list)




In [10]:
# beer_style_list=[]
# rating_list=[]
# for item in b_item_list[::2]:
#     beer_style_list.append(item)    
# # len(beer_style_list)

# for item in b_item_list[1::2]:
#     rating_list.append(item)
# len(rating_list)

In [38]:
# len(beer_style_list)
# len(rating_list)

In [11]:
# df =pd.DataFrame({"beer_style": beer_style_list, 
#                  "match_beer_style": beer_sty_list,
#                  "review_url":beer_url_list,
#                  "beer_name": beer_name_list,
#                  "brewry_name": brewry_list,
#                  "rating": rating_list})
# df

In [12]:
# df.sort_values('rating', ascending = False)
# len(df)*.2

In [13]:
# for x in b_list:
#     ind_beers = re.sub(r'<b>', '', x)
#     print(ind_beers)
#     beer_dict = {"style_name": beer.text}
#     all_styles.append(beer_dict)
# all_style

In [ ]:
# # list for all beer styles from beer advocate
# all_styles = []
# base_url = "https://www.beeradvocate.com"
# # get a dict to hold style and link
# for x in target_indexes:
#     ind_beers = tds[x].find_all('a')
#     print(ind_beers)
# #     for beer in ind_beers:
# #         beer_text = re.sub(r'\W+', '', beer.text.lower())
# #         beer_dict = {"style_name": beer.text, "match_name": beer_text}
# #         all_styles.append(beer_dict)

#  # Iterate through each book
# #  for beer in tds:
# #      # Use Beautiful Soup's find() method to navigate and retrieve attributes
# #      b = beer.find('b')
# # #      link = td.find('a')
# # #      href = link['href']
# # #      b = link['b']
# #      print('-----------')
# #      print(b)
# # #      print('http://books.toscrape.com/' + href)

In [ ]:
# # Iterate through all pages
# for x in range(5):
#     # beer advocate styles url
#     url = "https://www.beeradvocate.com/user/beers/?ba=brutaltruth"

#     # response from site
#     response = requests.get(url)

#     #create Beautiful soup object and parse
#     soup = BeautifulSoup(response.text, 'html.parser')
#     results = soup.find_all('table')

#     # find all 'tds' in results, only index 0 holds correct table
#     tds = results[0].find_all('td')
#     tds
 
    
#      # Iterate through each book
# #      for beer in tds:
# #          # Use Beautiful Soup's find() method to navigate and retrieve attributes
# #          b = beer.find('b')
#     #      link = td.find('a')
#     #      href = link['href']
#     #      b = link['b']
# #          print('-----------')
# #          print(b)
#     #      print('http://books.toscrape.com/' + href)

# # Click the 'Next' button on each page
# # browser.click_link_by_partial_text('next')

In [4]:
import re
re.findall(r"([\d.]*\d+)", '+3.4%')

['3.4']